In [2]:
import os
import pandas
import gc
import tqdm
import numpy
import time

In [3]:
root = '../resource/csv/'
class table:
    
    start = time.time()
    transaction = pandas.read_csv(os.path.join(root, "transactions_train.csv"), dtype={'article_id':str})
    submission = pandas.read_csv(os.path.join(root, "sample_submission.csv"))
    article = pandas.read_csv(os.path.join(root, "articles.csv"))
    customer = pandas.read_csv(os.path.join(root, "customers.csv"))
    end = time.time()
    print('elapsed {} time'.format(end-start))
    pass


elapsed 45.05496311187744 time


##  Transaction 

In [5]:
print("the first 6 data")
print('data size {}'.format(table.transaction.shape))
table.transaction.head()

the first 6 data
data size (31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [6]:
print("column [t_dat] between {} and {}".format(table.transaction['t_dat'].min(), table.transaction['t_dat'].max()))
print('column [customer_id] have {} unique user'. format(table.transaction['customer_id'].nunique()))
print('column [article_id] have {} unique article'. format(table.transaction['article_id'].nunique())) 
print("column [price] summary")
print(table.transaction['price'].describe()[1:])
print('column [sales_channel_id] have {} unique sale channel'.format(table.transaction['sales_channel_id'].nunique())) 

column [t_dat] between 2018-09-20 and 2020-09-22
column [customer_id] have 1362281 unique user
column [article_id] have 104547 unique article
column [price] summary
mean    0.027829
std     0.019181
min     0.000017
25%     0.015814
50%     0.025407
75%     0.033881
max     0.591525
Name: price, dtype: float64
column [sales_channel_id] have 2 unique sale channel


##  Article

In [ ]:
print("the first 6 data")
print('data size {}'.format(len(table.article)))
table.article.head()

##  Submission

In [7]:
print("the first 6 data")
print('data size {}'.format(len(table.submission)))
table.submission.head()

the first 6 data
data size 1371980


,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [8]:
table.submission['customer_id'].nunique() == table.transaction['customer_id'].nunique()

False

##  總結

1. 對於每一個 user 要預測一組對應的序列，由於評分標準針對 ground trouth 少於 12 的資料不會特別扣分，所以在策略以推薦 12 種商品較佳。
2. 有一些商品是沒有出現在資料 transaction 表中，但在資料 article 表中有出現。
3. 觀察資料 submission 表以及資料 transaction 表，發現有一些用戶是不存在於資料 transaction 表，但存在於 submission 表中，可以懷疑測試集是根據時間去切割，才有機會產生這樣的問題。

---

##  基礎策略

根據提交 submission 表的方式來建立學習資料，將 transaction 做初步轉換，建立一張名為 user 的表。
轉換之後，想知道幾件事情，首先是對於每一位用戶來說，他的購買紀錄中，購買最多次的商品是哪個？購買了幾次？
考慮所有商品的情況下，每個用戶的購買紀錄各是幾次？這有助於讓我們更理解用戶行為。
其中有人（不知道是不是一個人）購買某個商品次數高達 570 次，牛逼。
平均每個人購買次數是 23 次，這數量也是挺多的。

In [ ]:
from collections import Counter
# table.transaction['article_id'] = [str(i) for i in table.transaction['article_id']]
table.user = table.transaction.groupby('customer_id')['article_id'].apply(' '.join).reset_index()
table.user['max_article'] = [Counter(i.split(" ")).most_common(1)[0][0] for i in table.user['article_id']]
table.user['max_article_number'] = [Counter(i.split(" ")).most_common(1)[0][1] for i in table.user['article_id']]
table.user['purchase_number'] = [len(i.split(" ")) for i in table.user['article_id']]
# table.user.to_csv(os.path.join('/kaggle/working', 'user.csv'), index=False)
print("每位用戶購買最多次的商品數量")
print(table.user['max_article_number'].describe()[1:])
print("每位用戶所以購買次數統計")
print(table.user['purchase_number'].describe()[1:])

因為提交是針對未來七天的預測，對於特徵資料的建構應該是以用戶當作 row ，時間以天為單位當作 column 來建構。

In [4]:
table.history = table.transaction.groupby(['customer_id', 't_dat'])['article_id'].apply(" ".join).reset_index()
table.history = table.history.pivot_table(values='article_id', index='customer_id', columns='t_dat', aggfunc='first')
table.history = table.history.fillna("-1")
table.history = table.history.reset_index().rename_axis(None, axis=1)
table.history['seq'] = table.history.iloc[:,1:].apply(lambda x: "|".join(x), 1)
table.history.head()

,customer_id,2018-09-20,2018-09-21,2018-09-22,2018-09-23,2018-09-24,2018-09-25,2018-09-26,2018-09-27,2018-09-28,...,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,seq
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-1,0583558001 0639677008 0640244003,-1,-1,-1,0521269001 0666448006,-1,0583558001 0673677002,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1|0583558001 0639677008 0640244003|-1|-1|-1|0...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001 0541518023,-1,-1,-1,0663713001,-1,-1,-1,-1,...,-1,0794321007,-1,-1,-1,-1,-1,-1,-1,0663713001 0541518023|-1|-1|-1|0663713001|-1|-...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-...


In [5]:
table.history.to_csv('history.csv', index=False)

In [27]:
table.history.head()

0    -1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-...
1    -1|0583558001 0639677008 0640244003|-1|-1|-1|0...
2    0663713001 0541518023|-1|-1|-1|0663713001|-1|-...
3    -1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-...
4    -1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-1|-...
dtype: object

In [22]:
a['customer_id']

0    00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1    0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2    000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3    00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4    00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...
Name: customer_id, dtype: object

{'-1',
 '0568601006 0568601006',
 '0568601043',
 '0607642008 0745232001',
 '0625548001 0176209023 0627759010',
 '0656719005',
 '0694736004 0785710001 0812683013',
 '0697138006',
 '0785186005',
 '0797065001 0797065001',
 '0841260003 0887593002 0890498002 0795440001 0859416011'}

In [ ]:
table.history = table.history.rename_axis(None, axis=1)

In [ ]:
table.history = table.history.fillna("-1")

In [ ]:
table.history['sequence'] = list(table.history.iloc[:,1:].apply(lambda x: "|".join(x), 1).reset_index(drop=True))

In [ ]:
table.history.head()

,customer_id,2018-09-20,2018-09-21,2018-09-22,2018-09-23,2018-09-24,sequence
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-1,0583558001 0639677008 0640244003,-1,-1,-1,-1|0583558001 0639677008 0640244003|-1|-1|-1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001 0541518023,-1,-1,-1,0663713001,0663713001 0541518023|-1|-1|-1|0663713001
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004 0685687003 0685687004 0685687001 05...,-1,-1,0508184022 0522992001 0605106001 0567618001,-1,0505221004 0685687003 0685687004 0685687001 05...
3,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,-1,-1,-1,-1,0664180001,-1|-1|-1|-1|0664180001
4,0003abe64294e66a6310c3436fa9e5b754cc5603deef4f...,-1,-1,0640174004 0501820008 0543054005 0591334019,-1,-1,-1|-1|0640174004 0501820008 0543054005 0591334...


In [ ]:
# import tqdm
# a = table.transaction.loc[[i in list(table.time.loc[table.time['end']<"2019-12-31"]['customer_id']) for i in tqdm.tqdm(table.transaction['customer_id'])]]
# # table.transaction.loc[]

# # table.time.loc[table.time['end']<="2019-12-31"].shape

In [ ]:
# table.time['start'].min()

In [ ]:
# table.time['end'].max()

In [ ]:
# table.time.head()